In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import tsfel


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/khangphan/.pyenv/versions/3.10.5/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/khangphan/.pyenv/versions/3.10.5/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/khangphan/.pyenv/versions/3.10.5/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/khangphan/.pyenv/versions/3.10.5/lib/python3.10/site-packages/traitlets/config/appl

AttributeError: _ARRAY_API not found

In [2]:
files=os.listdir('./Raw_data')
# files.remove('processing.ipynb') 
# files.remove('.ipynb_checkpoints')
# if not macos/linux comment the. following code
try:
    files.remove('.DS_Store')
except:
    pass
# 
files

['s6_E1_A1.csv',
 's7_E1_A1.csv',
 's10_E1_A1.csv',
 's9_E1_A1.csv',
 's4_E1_A1.csv']

In [3]:
names=['sensor_'+str(i+1) for i in range(16)]
names.append('label')
df=pd.read_csv('./Raw_data/s6_E1_A1.csv',names=names)
df

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,label
0,6,-1,-2,-2,0,-1,-1,-1,-2,-1,-1,1,0,-1,-2,0,0
1,1,1,0,0,0,-2,-3,2,-2,0,-2,0,1,1,-1,1,0
2,1,-1,-1,0,-1,0,0,-1,-3,-1,-2,0,0,-2,-1,-2,0
3,6,0,-1,-2,-2,-4,1,0,-1,0,-1,0,1,1,-1,-3,0
4,3,-2,-1,0,-2,-1,-1,-1,0,-2,-1,-1,-1,-2,-2,-3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151788,-1,-2,0,-4,0,-2,-3,-4,-4,-1,-1,-1,0,-2,-1,-4,0
151789,-5,4,1,-7,-2,-6,-1,0,-2,-3,-3,-2,-1,-1,0,0,0
151790,4,-4,-4,-3,0,0,0,1,-2,-1,0,-2,-1,0,-2,-7,0
151791,0,-7,1,8,-1,-1,-1,0,1,0,-1,-2,-3,-1,-1,-1,0


In [4]:
import numpy as np
import pandas as pd
import tsfel
import numpy as np
from tsfel.utils.add_personal_features import add_feature_json
# --- Step 3: Set up config ---
personal_dir='./tsfel_feat.json'
add_feature_json('./custom_features.py',personal_dir)
cfg = tsfel.get_features_by_domain('statistical', json_path=personal_dir)
# cfg['statistical'].pop('Kurtosis', None)
# cfg['statistical'].pop('Kurtosis', None)

cfg

Feature Mode was added.
Feature AUC_abs was added.


{'statistical': {'AUC_abs': {'complexity': 'log',
   'description': '',
   'function': 'AUC_abs',
   'parameters': {'fs': 100},
   'use': 'yes'},
  'Absolute energy': {'complexity': 'log',
   'description': 'Computes the absolute energy of the signal.',
   'function': 'tsfel.abs_energy',
   'n_features': 1,
   'parameters': '',
   'tag': 'audio',
   'use': 'yes'},
  'Average power': {'complexity': 'constant',
   'description': 'Computes the average power of the signal.',
   'function': 'tsfel.average_power',
   'n_features': 1,
   'parameters': {'fs': 100},
   'tag': 'audio',
   'use': 'yes'},
  'ECDF': {'complexity': 'log',
   'description': 'Computes the values of ECDF (empirical cumulative distribution function) along the time axis.',
   'function': 'tsfel.ecdf',
   'n_features': 'd',
   'parameters': {'d': 10},
   'use': 'yes'},
  'ECDF Percentile': {'complexity': 'log',
   'description': 'Determines the percentile value of the ECDF.',
   'function': 'tsfel.ecdf_percentile',
   'n_

In [7]:
for i in [0]:
    for file in files:
        if not file.endswith('csv'):
            continue
        print(file)
        df=pd.read_csv(f'./Raw_data/{file}',names=names)
        df_transform = tsfel.time_series_features_extractor(cfg,df, fs=200
                                                            ,window_size=100,verbose=0
                                                            ,overlap=i,features_path='custom_features.py')
        df_transform=df_transform.rename(columns={'label_Mode': 'Target'})
        df_final=df_transform.loc[:,df_transform.columns.str.contains('|'.join(['Entropy','Target', 'Root mean square','Min','Max','Skew','Median','Mean','Interquartile range']),regex=True)]
    #     df_final=df_transform.loc[:,df_transform.columns.str.contains('|'.join(['Target', 'Root mean square','Min','Max','Skew','Median','Absolute energy','Kurtosis','Average power']),regex=True)]
        df_final=df_final.loc[:,~df_final.columns.str.contains('label',regex=True)]
        j=10*i
        try:
            os.makedirs(f'./processed_data1/{j}')
        except:
            pass
        df_final.to_csv(f'./processed_data1/{j}/{file}',index=False)
    

s6_E1_A1.csv



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<string>", line 1, in <module>
  File "/Users/khangphan/.pyenv/versions/3.10.5/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/khangphan/.pyenv/versions/3.10.5/lib/python3.10/multiprocessing/spawn.py", line 129, in _main
    return self._bootstrap(parent_sentinel)
  File "/Users/khangphan/.pyenv/versions/3.10.5/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/khangphan/.pyenv/versions/3.10.5/l

KeyboardInterrupt: 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<string>", line 1, in <module>
  File "/Users/khangphan/.pyenv/versions/3.10.5/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/khangphan/.pyenv/versions/3.10.5/lib/python3.10/multiprocessing/spawn.py", line 129, in _main
    return self._bootstrap(parent_sentinel)
  File "/Users/khangphan/.pyenv/versions/3.10.5/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/khangphan/.pyenv/versions/3.10.5/l

In [ ]:
df_final.shape